<h3>Week 3 Assignment on Segmenting and Clustering</h3>

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import csv
import urllib
import re
%matplotlib inline

In [2]:
#Extract table from wiki
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

tabs = soup.find_all('table', class_='wikitable sortable')
#print(tabs)
#print(tabs.prettify())

type(tabs)

bs4.element.ResultSet

<h3> Extract Data from Wikipedia</h3>

In [3]:
tabs = tabs[0]
type(tabs)

bs4.element.Tag

In [4]:
#EXTRACT DATA AND WRITE TO CSV FILE
with open('wikidata.csv', 'a') as csvFile:
    for row in tabs.find_all('tr'):
        writer = csv.writer(csvFile)
        writer.writerow(row)

In [5]:
#Convert csv to dataframe
datadf = pd.read_csv('wikidata.csv')
#datadf = datadf.drop(['.1', '.2'], axis=1)

<h3>Transform Data to Dataframe</h3>

In [6]:
datadf.head(10)

,,<th>Postcode</th>,.1,<th>Borough</th>,.2,<th>Neighbourhood </th>
0,\n,<td>M1A</td>,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
1,\n,<td>M2A</td>,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
2,\n,<td>M3A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Parkwoods"" title=""Parkwoods..."
3,\n,<td>M4A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Victoria_Village"" title=""Vi..."
4,\n,<td>M5A</td>,\n,"<td><a href=""/wiki/Downtown_Toronto"" title=""Do...",\n,"<td><a href=""/wiki/Harbourfront_(Toronto)"" tit..."
5,\n,<td>M5A</td>,\n,"<td><a href=""/wiki/Downtown_Toronto"" title=""Do...",\n,"<td><a href=""/wiki/Regent_Park"" title=""Regent ..."
6,\n,<td>M6A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Lawrence_Heights"" title=""La..."
7,\n,<td>M6A</td>,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Lawrence_Manor"" title=""Lawr..."
8,\n,<td>M7A</td>,\n,"<td><a href=""/wiki/Queen%27s_Park_(Toronto)"" t...",\n,<td>Not assigned\n</td>
9,\n,<td>M8A</td>,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>


In [7]:
#Replace columns with column headers
datadf.columns = ["one","Postcode","two", "Borough","three", "Neighbourhood"]

In [8]:
#Strip unwanted characters out of Postcode
datadf['Postcode'] = datadf['Postcode'].map(lambda x: x.lstrip('<td>').rstrip('</td>'))
datadf.head()

,one,Postcode,two,Borough,three,Neighbourhood
0,\n,M1A,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
1,\n,M2A,\n,<td>Not assigned</td>,\n,<td>Not assigned\n</td>
2,\n,M3A,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Parkwoods"" title=""Parkwoods..."
3,\n,M4A,\n,"<td><a href=""/wiki/North_York"" title=""North Yo...",\n,"<td><a href=""/wiki/Victoria_Village"" title=""Vi..."
4,\n,M5A,\n,"<td><a href=""/wiki/Downtown_Toronto"" title=""Do...",\n,"<td><a href=""/wiki/Harbourfront_(Toronto)"" tit..."


In [9]:
#Drop unwanted columns
datadf = datadf.drop(["one","two","three"], axis=1)

In [10]:
#"Clean column using Regular Expressions"
datadf['Borough'] = datadf['Borough'].str.extract(r'title="(.*?)">')
datadf.head()

,Postcode,Borough,Neighbourhood
0,M1A,NaN,<td>Not assigned\n</td>
1,M2A,NaN,<td>Not assigned\n</td>
2,M3A,North York,"<td><a href=""/wiki/Parkwoods"" title=""Parkwoods..."
3,M4A,North York,"<td><a href=""/wiki/Victoria_Village"" title=""Vi..."
4,M5A,Downtown Toronto,"<td><a href=""/wiki/Harbourfront_(Toronto)"" tit..."


In [11]:
#"Clean column using Regular Expressions"
datadf['Neighbourhood'] = datadf['Neighbourhood'].str.extract(r'title="(.*?)">')
datadf.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park (Toronto),NaN
9,M8A,NaN,NaN


In [12]:
#Remove null data from Borough Column
datadf = datadf[pd.notnull(datadf['Borough'])]

In [13]:
datadf.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [14]:
datadf.groupby(['Postcode'])

In [15]:
#Replace Null Neighbourhood data with Borough
datadf.loc[(pd.isnull(datadf.Neighbourhood), 'Neighbourhood')] = datadf.Borough
datadf.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront (Toronto)
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park (Toronto),Queen's Park (Toronto)
10,M9A,Etobicoke,Islington Avenue
11,M1B,"Scarborough, Toronto","Rouge, Toronto"
12,M1B,"Scarborough, Toronto","Malvern, Toronto"


In [17]:
datadf.shape

(416, 3)

<h3>Extracting Latitude and Longitude Data</h3>

In [18]:
#Convert csv to dataframe
latlongdf = pd.read_csv('Geospatial_Coordinates.csv')
latlongdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
latlongdf.columns = ["Postcode","Latitude","Longitdue"]
latlongdf.head()

,Postcode,Latitude,Longitdue
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
#Merge data from Geospatial data, spot check for accuracy
df_merge_col = pd.merge(latlongdf, datadf, on='Postcode')
df_merge_col.head(50)

,Postcode,Latitude,Longitdue,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,"Scarborough, Toronto","Rouge, Toronto"
1,M1B,43.806686,-79.194353,"Scarborough, Toronto","Malvern, Toronto"
2,M1B,43.806686,-79.194353,"Scarborough, Toronto","Rouge, Toronto"
3,M1B,43.806686,-79.194353,"Scarborough, Toronto","Malvern, Toronto"
4,M1C,43.784535,-79.160497,"Scarborough, Toronto",Highland Creek (Toronto)
5,M1C,43.784535,-79.160497,"Scarborough, Toronto",Rouge Hill
6,M1C,43.784535,-79.160497,"Scarborough, Toronto","Port Union, Toronto"
7,M1C,43.784535,-79.160497,"Scarborough, Toronto",Highland Creek (Toronto)
8,M1C,43.784535,-79.160497,"Scarborough, Toronto",Rouge Hill
9,M1C,43.784535,-79.160497,"Scarborough, Toronto","Port Union, Toronto"


In [22]:
df_merge_col.shape

(416, 5)